In [ ]:
import os
from data_partitioner_jsonl import *
from lisa_tagging import *

In [ ]:
file_name = 'corpus.jsonl'
dataset = "blogcorpus"

In [ ]:
%time
# input_path = os.path.join(f'/shared/3/projects/hiatus/idiolect/data/full_pilot/{dataset}', file_name)
# partition_dir = os.path.join(f'/shared/3/projects/hiatus/tagged_data/lisa_partitions/{dataset}', file_name.split('.')[0]) 
# tag_dir = os.path.join(f'/shared/3/projects/hiatus/tagged_data/lisa_partitions/tagged-{dataset}', file_name.split('.')[0])
# output_dir= f'/shared/3/projects/hiatus/tagged_data/lisa-{dataset}'
input_path = os.path.join(f'/shared/3/projects/hiatus/idiolect/data/full_pilot/{dataset}', file_name)
partition_dir = os.path.join(f'/shared/3/projects/hiatus/tagged_data/lisa_partitions/{dataset}', file_name.split('.')[0]) 
tag_dir = os.path.join(f'/shared/3/projects/hiatus/tagged_data/lisa_partitions/tagged-{dataset}', file_name.split('.')[0])
output_dir= f'/shared/3/projects/hiatus/tagged_data/lisa-{dataset}'


# ensures all directories will exist
for directory in [partition_dir, tag_dir, output_dir]:
    os.makedirs(directory, exist_ok=True)


## Partition the file into 50 parts

In [ ]:
# %%time
# partition_file(input_path, partition_dir, chunks=2)

In [ ]:
# from tokenization_enc_t5 import EncT5Tokenizer
# from modeling_enc_t5 import EncT5ForSequenceClassification
# import os
# import json
# import torch
# import time

# tokenizer = EncT5Tokenizer.from_pretrained("t5-base")
# model = EncT5ForSequenceClassification.from_pretrained(
#     "/shared/3/projects/hiatus/idiolect/models/stylegenome_lisa_sfam/lisa_checkpoint",
#     num_labels=768, problem_type="regression"
# )
# model.eval()


# def tag_lisa(obj):
#     global tokenizer, model
#     start_time = time.time()  # Start timing
#     try:
#         tokenized = tokenizer(
#             [obj['fullText']],
#             truncation=True, max_length=512, padding=True, return_tensors="pt"
#         )
#         with torch.no_grad():
#             prediction = model.forward(**tokenized)[0][0].cpu().float()
#         vector = torch.clamp(prediction, min=0.0, max=1.0).numpy().tolist()
#         if 'encodings' in obj:
#             del obj['encodings']
#         obj['lisa_vector'] = vector
#         print('tagged file')
#     except Exception as e:
#         print(f"Error processing object {obj['documentID']}: {e}")
#     end_time = time.time()  # End timing
#     elapsed_time = end_time - start_time  # Calculate elapsed time
#     print(f"Time taken to tag document {obj['documentID']}: {elapsed_time:.4f} seconds")  # Print the elapsed time
#     return obj

# def tag_partition(input_file, output_file):
#     print(f"Tagging file {input_file} with worker {os.getpid()}")
#     tagged_objects = []

#     try:
#         with open(input_file, 'r') as reader:
#             for line in reader:
#                 obj = json.loads(line.strip())
#                 tagged_object = tag_lisa(obj)
#                 tagged_objects.append(tagged_object)
# #                 print(f"Tagged object {obj['documentID']} for worker {os.getpid()}. Count: {len(tagged_objects)}")

#                 if len(tagged_objects) % 10 == 0:
# #                     print(f"Appending chunk of 10 objects to {output_file}")
#                     append_chunk(output_file, tagged_objects)
#                     tagged_objects = []

#         if tagged_objects:
#             append_chunk(output_file, tagged_objects)
#     except Exception as e:
#         print(f"Error tagging partition {input_file}: {e}")

# def append_chunk(output_file, chunk):
#     try:
#         with open(output_file, 'a') as writer:
#             for obj in chunk:
#                 writer.write(json.dumps(obj) + '\n')
#     except Exception as e:
#         print(f"Error appending chunk to {output_file}: {e}")
        
# tag_partition('/shared/3/projects/hiatus/tagged_data/lisa_partitions/blogcorpus/corpus/partition-1.jsonl', 'test_tagged.jsonl')



## Tag each partition with 1 CPU 

Set nice value low so we don't hog the server


In [ ]:
%%time
import logging
original_level = logging.getLogger().getEffectiveLevel()
logging.getLogger().setLevel(logging.ERROR)

tag_partitions(
               input_directory=partition_dir,
               output_directory=tag_dir,
               num_workers=2,
               )
logging.getLogger().setLevel(original_level)


## Join the partitioned files

In [ ]:
# join_tagged_files(input_directory=tag_dir,
#                   output_file=os.path.join(output_dir, file_name))

## Delete the temp directories

In [ ]:
# delete_partitioned_files(partition_dir)
# delete_partitioned_files(tag_dir)